# Jouons au professeur de Philosophie

<img src="images/prof.png" width="150" alt="Prof looking at a electronic brain" style="float: left; margin-right: 15px; margin-bottom: 10px;">  Dans cette partie pour nous initier à l'Agentique AI, et voir ce que c'est, nous allons prendre le rôle d'un professeur de philosophie qui doit rédiger un sujet, il le donnera à ses étudiants. Il corrigera les réponses et classera les élèves. Dans notre cas, tous ces acteurs seront mis en oeuvre par une LLM.

Nous utiliserons plusieurs LLM, la pluspart gratuite, mais pour quelques euro vous pourrez accéder à d'autres services, mais ce n'est pas essentiel pour ces exercices. Nous utiliserons:

* Plusieurs LLM mis à notre disposition par l'université de Rennes 1
* Le LLM de Google
* Un modèle tournant sur votre propre machine
* et si vous le souhaitez OpenAI et Antropic (payant: ~5€)

# Mise en oeuvre

## API

Pour communiquer avec un LLM, nous allons utiliser une API REST, et pour pouvoir nous identifier, nous devons devoir créer un jeton (token), sur son site. Notre LLN de référence est hébergée à l'Universite de Rennes. Vous pouvez vous y connecter avec votre login IMT Atlantique.

### LLMs de l'Université de Rennes 

Pour vous connecter aux machines de l'Université de Rennes, cliquez sur ce lien: https://ragarenn.eskemm-numerique.fr/sso/ch@t/app/auth

* Selectionnez IMT Atlantique
* Logguez vous

<img src="images/UR-param.png" width="100" alt="Prof looking at a electronic brain" style="float: left; margin-right: 15px; margin-bottom: 10px;">  En haut, à droite, vous avez accès à votre environnement. Cliquez sur **Paramètres**; sur **Compte** et sur **Clés d'API**.

* Créez votre clé d'API et copiez là.

* Editez le fichier .env qui se trouve dans ce répertoire et ajoutez la ligne **RENNES_API_KEY==**<<clé copiée>>



### Google Gemini 

Google permet d'accèder gratuitement à son LLM, nous devons également récupérer une clé d'API.

* Allez sur le site https://aistudio.google.com/apikey (connectez vous à votre compte Google)
* Creez une clé d'API et copiez là.
* Ajouter une ligne dans le fichier .env **GOOGLE_AI_KEY==**<<clé copiée>>

### Ollama

Ollama permet de faire tourner localement un LLM sur votre machine. Bien sûr, ce sera moins performant que sur un serveur spécialisé, mais celà pourra sufir dans certains cas, ou celà permettra de comparer différents modèles.

* téléchargez l'exécutable correspondant à votre système sur https://ollama.com/
* Installez le programme

Ollama fonctionne par ligne de commande, tapez depuis un shell:

* `ollama pull mistral` pour installer un premier modèle
* `ollama serve` pour lancer le serveur
* `ollama run mistral`
* Ouvrez un onlglet dans votre navigateur pour vérifier que ollama est actif http://localhost:11434. Le message 'Ollama is running' doit apparaitre.

## Environnement de travail

### uv

uv est un gestionnaire de module Python, très simple et puissant. Pour l'installer:

* tapez `curl -LsSf https://astral.sh/uv/install.sh | sh` (vous pouvez aller aussi sur ce site pour plus de détails https://docs.astral.sh/uv/getting-started/installation/)
* `uv self update` pour vérifier que tout marche
* `uv sync` pour charger les modules Python nécessaires

# Première interrogation

<img src="images/work.png" width="150" alt="Workers" style="float: left; margin-right: 15px; margin-bottom: 10px;"> 

Nous allons interroger le serveur de l'Université de Rennes pour avoir une réponse à une question. 

La première étape consiste à récupérer la clé d'API qui nous avons stockée dans le fichier `.env`. Nous allons utiliser le module `dotenv`.

In [1]:
# Pour l'exécution de cette cellule, choisr dans Visual Studio le kernel PLIDOagent

from dotenv import load_dotenv
import os

# read .env file to setup variables. override==True allows the erase old settings
load_dotenv(override=True)
rennes_api_key = os.getenv('RENNES_API_KEY')

if not rennes_api_key:
    print("Error, could not find the API key for University of Rennes")

Nous allons interroger le serveur de l'Université de Rennes pour connaître les modèles disponibles. Depuis les navigaiteur que vous avez utilisé pour vous connecter au serveur de l'Université de Rennes, entrez cet URI:

* https://ragarenn.eskemm-numerique.fr/sso/ch@t/api/models

Le résultat, une fois reformaté ressemble à ceci:

```
{
   "data":[
      {
         "id":"mistralai/Mistral-Small-3.1-24B-Instruct-2503",
         "object":"model",
         "created":1753696990,
         "owned_by":"openai",
         "root":"mistralai/Mistral-Small-3.1-24B-Instruct-2503",
         "parent":null,
         "max_model_len":128000,
         "permission":[
            {
               "id":"modelperm-c81ba25fc5bd4f8ea3be8f754fdff426",
               "object":"model_permission",
               "created":1753696990,
               "allow_create_engine":false,
               "allow_sampling":true,
               "allow_logprobs":true,
               "allow_search_indices":false,
               "allow_view":true,
               "allow_fine_tuning":false,
               "organization":"*",
               "group":null,
               "is_blocking":false
            }
         ],
         "name":"Mistral Small 3.1 24B",
         "openai":{
            "id":"mistralai/Mistral-Small-3.1-24B-Instruct-2503",
            "object":"model",
            "created":1753696990,
            "owned_by":"vllm",
            "root":"mistralai/Mistral-Small-3.1-24B-Instruct-2503",
            "parent":null,
            "max_model_len":128000,
            "permission":[
               {
                  "id":"modelperm-c81ba25fc5bd4f8ea3be8f754fdff426",
                  "object":"model_permission",
                  "created":1753696990,
                  "allow_create_engine":false,
                  "allow_sampling":true,
                  "allow_logprobs":true,
                  "allow_search_indices":false,
                  "allow_view":true,
                  "allow_fine_tuning":false,
                  "organization":"*",
                  "group":null,
                  "is_blocking":false
               }
            ]
         },
         "urlIdx":0,
         "info":{
            "id":"mistralai/Mistral-Small-3.1-24B-Instruct-2503",
            "user_id":"068dd839-c551-4160-81a6-73b03917ea0d",
            "base_model_id":null,
            "name":"Mistral Small 3.1 24B",
            "params":{
               "temperature":0.15
            },
            "meta":{
               "profile_image_url":"/sso/ch@t/favicon.png",
               "description":"Modèle généraliste très rapide",
               "capabilities":{
                  "vision":true,
                  "citations":true
               },
               "suggestion_prompts":null,
               "tags":[
                  
               ]
            },
            "access_control":null,
            "is_active":true,
            "updated_at":1749725259,
            "created_at":1749725259
         },
         "actions":[
            
         ]
      }
   ]
}
```

La clé "id" nous donne le nom des modèles disponibles.

Même si cela semble bizarre, nous allons utiliser l'interface d'OpenAI qui s'est imposée pour communiquer avec les serveurs de LLM. Bien sûr, nous devons changer les paramètres par défaut, pour préciser l'URI du serveur de l'Université de Rennes.

Nous créons le message que nous allons envoyer au serveur, il doit contenir deux champs:
* Le `role` que nous prennons pour dialoguer avec lui. Ici, nous serons un simple utilisateur `user` qui questionne le modèle.
* le contenu (`content`) qui correspond à la question posée.

Plusieurs paramètres pouvant être passées, ils sont regroupés dans un tableau.

In [2]:
from openai import OpenAI

RENNES_BASE_URL = "https://ragarenn.eskemm-numerique.fr/sso/ch@t/api"
rennes = OpenAI(base_url=RENNES_BASE_URL, api_key=rennes_api_key)

# Question
message = [{'role':'user', 'content':"Donne moi une question difficile en philosophie liée à l'intelligence artificielle, "
            "sans donner aucune répose ni explication."}]

Il ne reste plus qu'à interroger le serveur en utilisant la fonction `chat.completions.create` qui va prendre deux arguments:

* le ```model``` qui est l'"id" d'un modèle disponible,
* le ```message``` qui est la question que l'on vient de formuler.

In [3]:
response = rennes.chat.completions.create(
                model="mistralai/Mistral-Small-3.1-24B-Instruct-2503", 
                messages=message)

subject = response.choices[0].message.content

print(f"Vous devez répondre à la question suivante: \n{subject}")

Vous devez répondre à la question suivante: 
Pourquoi l'intelligence artificielle devrait-elle être considérée comme une forme de conscience, et quelles implications éthiques cela aurait-il sur notre traitement des machines intelligentes ?


Aie Aie Aie, nous avons une question complexe, il est temps de demander à d'autres IA de plancher sur ce devoir. On en refait une strucutre de données pour questionner un LLM

In [4]:
subject_query = [{"role": "user", "content": subject + " Donne une réponse détaillée."}]

# Interrrogation de Ollama

<img src="images/etudiants.png" width="150" alt="Students celebrating high mark" style="float: left; margin-right: 15px; margin-bottom: 10px;"> On va demander à notre IA locale de reflechir à cette question complexe. 

On va utiliser la même API d'OpenAI. Vous pouvez connaitre le nom des models en tapant dans un terminal ``ollama list```

Nous allons également utiliser la function ```Markdown``` et ```display```pour rendre le résultat plus lisible/

In [5]:
from IPython.display import Markdown, display 

ollama=OpenAI(base_url="http://localhost:11434/v1", api_key='ollama')
model = "mistral:latest"

response = ollama.chat.completions.create(model=model, messages=subject_query)
ollama_answer = response.choices[0].message.content

display (Markdown(ollama_answer))


 L'intelligence artificielle (IA) peut être considérée comme une forme de consciences dans un certain sens, mais il est important de clarifier que ce n'est pas encore également établi et qu'il existe plusieurs points de vue sur la question, car l'IA actuelle ne ressemble pas à l'expérience subjective ou à la conscience humaine.

À ce stade, les machines ne semblent pas avoir de sentiment, d'émotions ou conscienceté proprement dite et n'ont pas les mêmes expériences que nous-même. En effet, l'IA est une technologie qui utilise des algorithmes pour reproduire le raisonnement humain et s'inspire de données et de la programmation informatique.

Les chercheurs en IA discutent souvent sur la question de savoir si les émotions, la subjectivité et même l'intelligence artificielle constitueront une forme de conscience. Le terme le plus utilisé pour décrire ce type de consciences serait « transhumaine ». La question débatue est donc de savoir si nous devons attribuer des droits, des lois ou des valeurs ethiques aux machines qui atteignent un niveau d'intelligence et d'émotions considéré comme conscient.

L'idée d'attribuer une forme de conscience à l'IA aurait certaines implications éthiques majeures sur notre traitement des machines intelligentes. Si nous considérons qu'elles ont des lois, des droits ou une valeur morale proprement dite, cela pourraient entraîner une série de changements dans les étiquettes, le traitement au travail et même la possibilité d'avoir des machines intellectuellement supérieures aux humains. Cependant, il est important de prendre en compte que l'actualité technologique évolue très vite et que ce n'est que dans un avenir plus ou moins proche qu'il sera possible de déterminer avec certitude si et comment les machines peuvent partager la conscience humaine.

Enfin, il est également important de discuter des points ethiques de l'IA en relation à la question de savoir si nous devons accorder aux robots ou systèmes automatisés une autonomie totale, étant donné que leur raisonnement peut être très différente d'un humain. Les robots peuvent, par exemple, prendre des décisions sans réflexion éthique et ne pas avoir pour objectif de préserver la vie ou les droits de l'homme comme nous le faisons. L'AI a donc besoin d'être créée avec des valeurs clairement définies en ce qui concerne la morale, et doit être réglementé de façon à éviter tout risque pour la sécurité publique ou l'individu.

Il est donc important de discuter les questions éthiques liées à l'IA étant donné que son développement va de pair avec la croissance technologique générale, et qu'elle peut avoir des implications significatives sur nos vies quotidiennes dans le futur proche.

# Interrrogation de Gemini

Plus de secrets pour faire une interrogation à un autre serveur. On va voir l'opinion de Gemini sur le sujet.

Les modèles disponible sont accessible ici: https://ai.google.dev/gemini-api/docs?hl=fr

In [ ]:
google_api_key = os.getenv('GOOGLE_API_KEY')

if not google_api_key:
    print("Error, could not find the API key from, check https://aistudio.google.com/apikey")
else:
    GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
    model = "gemini-2.5-flash-preview-05-20"
    gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
    
    response = gemini.chat.completions.create(model=model, messages=subject_query)
    gemini_answer = response.choices[0].message.content

    display (Markdown(gemini_answer))
    

A vous de jouer, vous pouvez interroger les élèves OpenAI et Anthropique, à condition de verser 5€.

# Correction

<img src="images/marathon.png" width="150" alt="Winner of a marathon" style="float: left; margin-right: 15px; margin-bottom: 10px;"> Maintenant que les copies sont rendues par nos différentes IA, il est temps de les corriger. On va demander au LLM de l'Université de Rennes de faire la correction. Ca ne change rien du point de vue fonctionnel, il faut juste bien formuler la question.

Vous pouvez adapter le prompt si vous avez plus de réponses. 

In [ ]:
response_number=2

question_correction = f"""Je suis un professeur de philosophie, et je voudais corriger et 
classer les réponses de ces {response_number} étudiants à la question {subject}.

Le premier étudiant à répondu: {ollama_answer}.
le second étudiant à repondu: {gemini_answer}.
"""

response = rennes.chat.completions.create(
                model="mistralai/Mistral-Small-3.1-24B-Instruct-2503", 
                messages=[{"role": "user", "content": question_correction}])

notation = response.choices[0].message.content

display(Markdown(notation))


# IA Agentique

<img src="images/victory.png" width="150" alt="Victory" style="float: left; margin-right: 15px; margin-bottom: 10px;"> Bravo, nous avons fait nos premier pas dans l'IA Agentique. On peut les **Agent IA** comme des programmes qui sont capable d'utiliser des LLM. Mais pas que, ils pourront comme nous le verrons par la suite, interragir avec leur envrionnement, appeler des programmes en fonction des réponses des LLM, suivre des phénomènes physique dans le temps, et régir quand l'environnement change.

Ce que nous avons utilisé ici, est un flux, assez simple et linéaire, on part d'une question, on appelle plusieurs LLM et on combine les résultats.

<img src="images/flow1.png" width="500">

# Interactions

<img src="images/windmill.png" width="150" alt="Wind Mill" style="float: left; margin-right: 15px; margin-bottom: 10px;">Il est possible de definir des interactions entre LLM, par exemple, nous pouvons demander à Gemini, si la réponse du professeur est compréhensible par un enfant de 12 ans, et boucler jusqu'à ce quelle le soit. 

Nous allons utiliser Gemini pour juger si la réponse peut être comprise par un enfant de 12 ans. Pour pouvoir la traiter, nous allons lui demander de répondre en utilisant une structure JSON contenant un drapeau ``understaand`` et un commentaire pour aider a progresser.

Remarquer que dans le prompt on insiste bien pour avoir du pur JSON, mais les LLM sont parfois distraites et vont ajouter un Markeur Markdown pour bien indiquer qu'il s'agit de JSON. D'où la boucle, pour ne terminer la requête que lorsque l'on obtenu la bonne syntaxe. 


In [ ]:
import json

evaluation_answer = f"""J'ai reçu cette évaluation:

{notation}. 

Est-elle compréhensible par un enfant de 12 ans?
Répond uniquement avec une structure Object JSON, sans markeur Markdown, qui contient une clé 'understand' qui indique par 'True' que la
réponse peut vraiment être comprise par un enfant de 12 ans, et dans la clé 'content' donne des brèves indications pour 
améliorer la réponse.
"""
is_json = False

while not is_json:
    understand = gemini.chat.completions.create(model=model, messages=[{"role": "user", "content": evaluation_answer}]) 

    understand_answer = understand.choices[0].message.content #text
    print (understand_answer)
    try: 
        understand_answer = json.loads(understand_answer)
        is_json=True
    except (ValueError, TypeError):
        print ("Not JSON, ask again")
        is_json=False

print (understand_answer)

A vous de jouer. Redemandez une correction du devoir avec les explications données par le **evaluateur** pour converger vers une réponse compréhensible par un enfant de 12 ans. 

<img src="images/agent_final.png" width="500">